In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras.regularizers import l2


In [2]:
class FM_Layer(Layer):
    def __init__(self, feature_columns,k,w_reg=1e-6, v_reg=1e-6):

        super(FM_Layer, self).__init__()
        self.sparse_feature_columns = feature_columns
        self.index_mapping = []
        self.feature_length = 0
        for feat in self.sparse_feature_columns:
            self.index_mapping.append(self.feature_length)
            self.feature_length += feat['feat_num']
        self.k = k
        self.w_reg = w_reg
        self.v_reg = v_reg
        
        
    def build(self, input_shape):
        self.w0 = self.add_weight(name="w0", shape=(1,), initializer=tf.zeros_initializer(), trainable=True)
        self.w = self.add_weight(name='w', shape=(self.feature_length,1),
                                initializer=tf.random_normal_initializer(),
                                regularizer=l2(self.w_reg),trainable=True)
        self.V = self.add_weight(name='V', shape= (self.feature_length, self.k), initializer=tf.random_normal_initializer(),
                                regularizer=l2(self.v_reg),trainable=True)
        
    def call(self,inputs,**kwargs):
        inputs = inputs + tf.convert_to_tensor(self.index_mapping)
        first_order = self.w0 + tf.reduce_sum(tf.nn.embedding_lookup(self.w, inputs), axis=1) #(batch_size,1)
        second_inputs = tf.nn.embedding_lookup(self.V,inputs) #(batch_size, 1, embed_dim)
        square_sum = tf.square(tf.reduce_sum(second_inputs, axis=1,keepdims=True))  # (batch_size, 1, embed_dim)
        sum_square = tf.reduce_sum(tf.square(second_inputs), axis=1,keepdims=True) #(batch_size,1, embed_dim)
        second_order = 0.5*tf.reduce_sum(square_sum-sum_square, axis = 2) #(batch_size,1)
        outputs = first_order + second_order
        return outputs
        
    
class FM(Model):
    
    def __init__(self, feature_columns, k, w_reg=1e-6,v_reg=1e-6):
        super(FM, self).__init__()
        self.sparse_feature_columns = feature_columns
        self.fm = FM_Layer(feature_columns, k, w_reg, v_reg)
            
    def call(self, inputs, **kwargs):
        fm_outputs = self.fm(inputs)
        outputs = tf.nn.sigmoid(fm_outputs)
        return outputs
    def summary(self,**kwargs):
        sparse_inputs = Input(shape=(len(self.sparse_feature_columns),),dtype=tf.int32)
        Model(inputs=sparse_inputs,outputs=self.call(sparse_inputs)).summary()

           

In [4]:
import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

import sys
sys.path.append('..')
from data_process.criteo import create_criteo_dataset

import datetime
 


[{'feat_name': 'C1', 'feat_num': 541, 'embed_dim': 8}, {'feat_name': 'C2', 'feat_num': 497, 'embed_dim': 8}, {'feat_name': 'C3', 'feat_num': 43870, 'embed_dim': 8}, {'feat_name': 'C4', 'feat_num': 25184, 'embed_dim': 8}, {'feat_name': 'C5', 'feat_num': 145, 'embed_dim': 8}, {'feat_name': 'C6', 'feat_num': 12, 'embed_dim': 8}, {'feat_name': 'C7', 'feat_num': 7623, 'embed_dim': 8}, {'feat_name': 'C8', 'feat_num': 257, 'embed_dim': 8}, {'feat_name': 'C9', 'feat_num': 3, 'embed_dim': 8}, {'feat_name': 'C10', 'feat_num': 10997, 'embed_dim': 8}, {'feat_name': 'C11', 'feat_num': 3799, 'embed_dim': 8}, {'feat_name': 'C12', 'feat_num': 41312, 'embed_dim': 8}, {'feat_name': 'C13', 'feat_num': 2796, 'embed_dim': 8}, {'feat_name': 'C14', 'feat_num': 26, 'embed_dim': 8}, {'feat_name': 'C15', 'feat_num': 5238, 'embed_dim': 8}, {'feat_name': 'C16', 'feat_num': 34617, 'embed_dim': 8}, {'feat_name': 'C17', 'feat_num': 10, 'embed_dim': 8}, {'feat_name': 'C18', 'feat_num': 2548, 'embed_dim': 8}, {'feat_n

In [5]:
file = '../data/Criteo/train.txt'
read_part = True
sample_num = 5000000
test_size = 0.2

k = 8

learning_rate = 0.001
batch_size = 4096
epochs = 20


In [6]:
feature_columns,(train_X,train_y),(test_X,test_y) = create_criteo_dataset("../data/Criteo/train.txt")

In [8]:
model = FM(feature_columns=feature_columns, k=k)
model.summary()
model.compile(loss=binary_crossentropy,optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

t1 = datetime.datetime.now()
model.fit(train_X,train_y,epochs=epochs,
         callbacks=[EarlyStopping(monitor="val_loss", patience=2)],
         batch_size=batch_size,
          validation_split=0.1
         ) 
t2 = datetime.datetime.now()
print("时间为:%d" %((t2-t1).total_seconds()))
print("test AUC:  %f" %model.evaluate(test_X,test_y,batch_size=batch_size)[1])


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 39)]              0         
_________________________________________________________________
fm__layer_1 (FM_Layer)       (None, 1)                 2183743   
_________________________________________________________________
tf_op_layer_Sigmoid_1 (Tenso [(None, 1)]               0         
Total params: 2,183,743
Trainable params: 2,183,743
Non-trainable params: 0
_________________________________________________________________
Train on 72000 samples, validate on 8000 samples
Epoch 1/20


/home/xindun/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


72000/72000 [==============================] - 1s 15us/sample - loss: 0.6193 - auc_1: 0.5365 - val_loss: 0.5549 - val_auc_1: 0.5848
Epoch 2/20
72000/72000 [==============================] - 0s 2us/sample - loss: 0.5216 - auc_1: 0.6697 - val_loss: 0.5170 - val_auc_1: 0.6434
Epoch 3/20
72000/72000 [==============================] - 0s 2us/sample - loss: 0.4824 - auc_1: 0.7705 - val_loss: 0.4978 - val_auc_1: 0.6959
Epoch 4/20
72000/72000 [==============================] - 0s 2us/sample - loss: 0.4465 - auc_1: 0.8225 - val_loss: 0.4829 - val_auc_1: 0.7231
Epoch 5/20
72000/72000 [==============================] - 0s 2us/sample - loss: 0.4114 - auc_1: 0.8570 - val_loss: 0.4740 - val_auc_1: 0.7371
Epoch 6/20
72000/72000 [==============================] - 0s 2us/sample - loss: 0.3793 - auc_1: 0.8820 - val_loss: 0.4692 - val_auc_1: 0.7437
Epoch 7/20
72000/72000 [==============================] - 0s 2us/sample - loss: 0.3498 - auc_1: 0.9021 - val_loss: 0.4675 - val_auc_1: 0.7483
Epoch 8/20
72000